In [1]:
# Bring in (new) PNC cohort

import pickle
from pathlib import Path
import numpy as np

newdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC'
newdemo = pickle.load(open(f'{newdir}/demographics.pkl', 'rb'))

newfc = dict()
age = []
sex = []
race = []

for sub in newdemo['age_at_cnb']:
    for task in ['rest', 'nback', 'emoid']:
        f = f'{newdir}/fc/{sub}_task-{task}_fc.npy'
        if not Path(f).exists():
            continue
        if sub not in newdemo['Race'] or newdemo['Race'][sub] not in ['AA', 'EA']:
            continue
        p = np.load(f)
        a = newdemo['age_at_cnb'][sub]
        s = newdemo['Sex'][sub] == 'M'
        r = newdemo['Race'][sub] == 'AA'
        newfc[f'{sub}-{task}'] = p
        age.append(a)
        sex.append(s)
        race.append(r)
        
age = np.array(age)
sex = np.array(sex)
race = np.array(race)

print(age.shape)
print(np.mean(age))
print(np.mean(sex))
print(np.mean(race))
print(len(newfc))

(3849,)
14.398285268901013
0.473629514159522
0.4811639386853728
3849


In [7]:
# Angle estimate

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

pi = 3.14
pi2 = 2*pi

def tonp(t):
    return t.detach().cpu().numpy()

class AngleBasis(nn.Module):
    def __init__(self, mixn):
        super(AngleBasis, self).__init__()
        self.mixn = mixn
        self.thetas = nn.Parameter((pi*torch.rand(self.mixn,264)+pi/2).float().cuda())
        self.jitter = nn.Parameter(torch.ones(self.mixn,264).float().cuda())
        
    def project(self):
        with torch.no_grad():
#             self.thetas[:,50] = pi # Baseline
#             if self.jitter[0,50] < 0.5:
#                 self.jitter[:,50] = 0.5
#             self.thetas[self.thetas < 0] = 0
#             self.thetas[self.thetas > pi2] = pi2
            self.jitter[self.jitter < 0] = 0
            self.jitter[self.jitter > 1] = 1
        
    def phases(self):
        t0 = self.thetas.unsqueeze(2)
        t1 = self.thetas.unsqueeze(1)
        return t0-t1
    
    def jit(self):
        j0 = self.jitter.unsqueeze(2)
        j1 = self.jitter.unsqueeze(1)
        return j0*j1
    
    def ps(self, jitter=True):
        t = self.phases()
        p = torch.cos(t)
        if jitter:
            j = self.jit()
            p = j*p
        return p
    
    def dump(self):
        return dict(mixn=self.mixn, thetas=tonp(self.thetas), jitter=tonp(self.jitter))
    
    def psum(self):
        return torch.mean(self.ps(), axis=0)
    
    def pvec(self):
        a,b = np.triu_indices(264,1)
        p = self.psum()
        return p[a,b]
    
print('Complete')

Complete


In [8]:
nepochs = 500
pperiod = 500

def fromnp(x):
    return torch.from_numpy(x).float().cuda()

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

bases = dict()
i = 0

for subtask in newfc:
    x = fromnp(newfc[subtask])

    min_loss = 10
    sav = None

    for _ in range(5):
        basis = AngleBasis(2)
        optim = torch.optim.Adam(basis.parameters(), lr=1e-2, weight_decay=0)

        for e in range(nepochs):
            optim.zero_grad()
            xhat = basis.pvec()
            loss = rmse(xhat, x)
            loss.backward()
            optim.step()
            basis.project()
            if e == nepochs-1 or e % pperiod == 1:
                print(f'{e} {float(loss)}')

        if loss < min_loss:
            print('Saving')
            min_loss = float(loss)
            sav = basis
    
    bases[subtask] = sav.dump()
    i += 1
    
    print(f'Finished {i} {subtask}')
        
print('Complete')

1 0.4962414801120758
499 0.11026391386985779
Saving
1 0.5026309490203857
499 0.1098570004105568
Saving
1 0.5020986795425415
499 0.11000295728445053
1 0.5079014897346497
499 0.11053413152694702
1 0.5024478435516357
499 0.10795178264379501
Saving
Finished 1 600001103037-rest
1 0.5152788162231445
499 0.10462649911642075
Saving
1 0.5197123289108276
499 0.10448174923658371
Saving
1 0.5144919753074646
499 0.10147969424724579
Saving
1 0.5141124725341797
499 0.1044643297791481
1 0.5175228714942932
499 0.10757289081811905
Finished 2 600001103037-nback
1 0.5494254231452942
499 0.11408665776252747
Saving
1 0.5458175539970398
499 0.12430064380168915
1 0.544775128364563
499 0.12982681393623352
1 0.5459489226341248
499 0.1197211742401123
1 0.5548510551452637
499 0.12734383344650269
Finished 3 600001103037-emoid
1 0.45118680596351624
499 0.08230143785476685
Saving
1 0.44819191098213196
499 0.08438418060541153
1 0.445565789937973
499 0.0800829827785492
Saving
1 0.44459018111228943
499 0.08338477462530

499 0.0840248242020607
Saving
1 0.4593971371650696
499 0.08609376847743988
1 0.46279627084732056
499 0.08524633944034576
1 0.4802708327770233
499 0.0808621495962143
Saving
Finished 31 600051187879-nback
1 0.4514892101287842
499 0.0822070986032486
Saving
1 0.4595552384853363
499 0.08458614349365234
1 0.4489486515522003
499 0.07922425121068954
Saving
1 0.4638127088546753
499 0.07857337594032288
Saving
1 0.46221688389778137
499 0.08281884342432022
Finished 32 600051187879-emoid
1 0.47815752029418945
499 0.09870074689388275
Saving
1 0.4884108304977417
499 0.09804056584835052
Saving
1 0.4877675771713257
499 0.09840567409992218
1 0.48475462198257446
499 0.09901202470064163
1 0.48456665873527527
499 0.09647088497877121
Saving
Finished 33 600052596653-rest
1 0.45268139243125916
499 0.08042852580547333
Saving
1 0.4448399543762207
499 0.08107591420412064
1 0.45164188742637634
499 0.08156616985797882
1 0.4560334086418152
499 0.0807296559214592
1 0.4489215612411499
499 0.08114587515592575
Finished

499 0.10329282283782959
Saving
1 0.4744037091732025
499 0.10370945185422897
Finished 61 600098941368-rest
1 0.45142999291419983
499 0.07867071777582169
Saving
1 0.44307681918144226
499 0.07900317758321762
1 0.4517781436443329
499 0.07777606695890427
Saving
1 0.45326924324035645
499 0.07829167693853378
1 0.466187983751297
499 0.0751902237534523
Saving
Finished 62 600098941368-nback
1 0.44095322489738464
499 0.08408688753843307
Saving
1 0.45286667346954346
499 0.08214785903692245
Saving
1 0.45710495114326477
499 0.08813166618347168
1 0.4545903503894806
499 0.08047795295715332
Saving
1 0.4542732536792755
499 0.08126923441886902
Finished 63 600098941368-emoid
1 0.4594873785972595
499 0.06064910069108009
Saving
1 0.4494064152240753
499 0.05854947865009308
Saving
1 0.4695620536804199
499 0.05769665911793709
Saving
1 0.4573311507701874
499 0.06088770180940628
1 0.47444719076156616
499 0.05786215886473656
Finished 64 600109657100-rest
1 0.43761146068573
499 0.07242918759584427
Saving
1 0.43767

499 0.11118209362030029
Saving
1 0.47591695189476013
499 0.11340867727994919
1 0.4740331768989563
499 0.11160551756620407
Finished 91 600167349259-rest
1 0.46214011311531067
499 0.08682133257389069
Saving
1 0.4642341136932373
499 0.08444777131080627
Saving
1 0.46981921792030334
499 0.08472777158021927
1 0.46585971117019653
499 0.08288346976041794
Saving
1 0.4640507102012634
499 0.088900625705719
Finished 92 600167349259-nback
1 0.4583083689212799
499 0.08733446896076202
Saving
1 0.4624726176261902
499 0.08791224658489227
1 0.449444055557251
499 0.0791429951786995
Saving
1 0.4527020752429962
499 0.08085598796606064
1 0.4584980309009552
499 0.08312415331602097
Finished 93 600167349259-emoid
1 0.48090529441833496
499 0.10177180916070938
Saving
1 0.48678919672966003
499 0.0991295725107193
Saving
1 0.4838985800743103
499 0.10313954204320908
1 0.4819526672363281
499 0.09787292778491974
Saving
1 0.47937503457069397
499 0.09986680001020432
Finished 94 600173623767-rest
1 0.4560258388519287
499

499 0.09065020084381104
Saving
1 0.4689103066921234
499 0.09294573217630386
1 0.4750140905380249
499 0.09475474804639816
1 0.47218507528305054
499 0.09491343796253204
Finished 121 600210241146-emoid
1 0.4621776044368744
499 0.09070947766304016
Saving
1 0.4633064866065979
499 0.09175765514373779
1 0.4709751009941101
499 0.09328942745923996
1 0.4652334749698639
499 0.09235124289989471
1 0.46913304924964905
499 0.09053628146648407
Saving
Finished 122 600210683444-rest
1 0.44558271765708923
499 0.08161424100399017
Saving
1 0.44517016410827637
499 0.08063972741365433
Saving
1 0.4474458396434784
499 0.08136595785617828
1 0.43267446756362915
499 0.08143064379692078
1 0.4370695650577545
499 0.0809142217040062
Finished 123 600210683444-nback
1 0.4821545481681824
499 0.08627980202436447
Saving
1 0.4782170057296753
499 0.08495201170444489
Saving
1 0.47447577118873596
499 0.08291732519865036
Saving
1 0.4790271818637848
499 0.08173801004886627
Saving
1 0.4699343740940094
499 0.08390311896800995
Fin

...

499 0.09144003689289093
1 0.49545109272003174
499 0.09313502162694931
1 0.49270153045654297
499 0.08944321423768997
Saving
1 0.4971003532409668
499 0.08990538120269775
Finished 1800 604754412893-nback
1 0.49715980887413025
499 0.09726186096668243
Saving
1 0.4998777508735657
499 0.09723297506570816
Saving
1 0.5012774467468262
499 0.0953066274523735
Saving
1 0.4962459206581116
499 0.09329351037740707
Saving
1 0.4992395341396332
499 0.09728419780731201
Finished 1801 604754412893-emoid
1 0.5061043500900269
499 0.10646265000104904
Saving
1 0.5020768642425537
499 0.10547702014446259
Saving
1 0.504870593547821
499 0.11515433341264725
1 0.5070395469665527
499 0.11075989902019501
1 0.5041812062263489
499 0.10336572676897049
Saving
Finished 1802 604770953198-rest
1 0.5069615840911865
499 0.10088346153497696
Saving
1 0.5141391158103943
499 0.10434217751026154
1 0.5092688202857971
499 0.10231934487819672
1 0.508924126625061
499 0.10682280361652374
1 0.5069356560707092
499 0.10377049446105957
Finis

499 0.091813825070858
Saving
1 0.4966321587562561
499 0.09739956259727478
1 0.4973437488079071
499 0.09122031182050705
Saving
1 0.49641212821006775
499 0.09769471734762192
Finished 1830 604851308167-emoid
1 0.44626563787460327
499 0.07435742765665054
Saving
1 0.43395760655403137
499 0.07690401375293732
1 0.4436917304992676
499 0.07536809891462326
1 0.43055588006973267
499 0.07530860602855682
1 0.4495794475078583
499 0.07667604088783264
Finished 1831 604853384855-nback
1 0.4857247769832611
499 0.07459217309951782
Saving
1 0.49663427472114563
499 0.06935133039951324
Saving
1 0.49580198526382446
499 0.06872376054525375
Saving
1 0.49197569489479065
499 0.07757852971553802
1 0.5046473145484924
499 0.06389322876930237
Saving
Finished 1832 604853384855-emoid
1 0.4365309178829193
499 0.08284240961074829
Saving
1 0.44054514169692993
499 0.08283726125955582
Saving
1 0.44709673523902893
499 0.08986645191907883
1 0.4486579895019531
499 0.0830826610326767
1 0.44576942920684814
499 0.083126083016395

499 0.068245068192482
Saving
1 0.5013705492019653
499 0.06876885890960693
1 0.47167056798934937
499 0.06688320636749268
Saving
1 0.4975661337375641
499 0.07586359232664108
1 0.4731084704399109
499 0.0701911598443985
Finished 1860 604911918083-nback
1 0.5101842284202576
499 0.06593269854784012
Saving
1 0.500490128993988
499 0.06732986867427826
1 0.4945073425769806
499 0.06861802935600281
1 0.5008187890052795
499 0.0641484335064888
Saving
1 0.4905124008655548
499 0.08083279430866241
Finished 1861 604911918083-emoid
1 0.483662486076355
499 0.10298977047204971
Saving
1 0.4853713810443878
499 0.09705132246017456
Saving
1 0.4829086661338806
499 0.09924288094043732
1 0.48132410645484924
499 0.10199083387851715
1 0.47992396354675293
499 0.1004585549235344
Finished 1862 604921668926-rest
1 0.5176098942756653
499 0.10325849801301956
Saving
1 0.5176430940628052
499 0.10582619905471802
1 0.5256123542785645
499 0.10430459678173065
1 0.5178071856498718
499 0.10249364376068115
Saving
1 0.518156588077

499 0.1055544912815094
Saving
1 0.4924270808696747
499 0.10781880468130112
1 0.4980193078517914
499 0.10150343179702759
Saving
1 0.5013008117675781
499 0.10301423817873001
1 0.4952743351459503
499 0.11256401985883713
Finished 1890 605034869897-rest
1 0.46129244565963745
499 0.08537664264440536
Saving
1 0.4653875529766083
499 0.08500662446022034
Saving
1 0.4667564928531647
499 0.08386164903640747
Saving
1 0.4659157693386078
499 0.08516030013561249
1 0.4653269648551941
499 0.08426667749881744
Finished 1891 605034869897-nback
1 0.4567767381668091
499 0.08589200675487518
Saving
1 0.46497923135757446
499 0.08172594010829926
Saving
1 0.46272769570350647
499 0.08325397968292236
1 0.4654633104801178
499 0.08509628474712372
1 0.46628665924072266
499 0.08056171983480453
Saving
Finished 1892 605034869897-emoid
1 0.45353347063064575
499 0.09018729627132416
Saving
1 0.45237091183662415
499 0.08790881186723709
Saving
1 0.45888572931289673
499 0.08709331601858139
Saving
1 0.45917826890945435
499 0.08

KeyboardInterrupt: 

In [9]:
import pickle

pickle.dump(bases, open('/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasis2.pkl', 'wb'))

print('Complete')

Complete
